# Imports

In [5]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import random_split
from PIL import Image
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch


In [19]:
data_dir = "../dataset_regular_or_sun"

transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor()
])

dataset = datasets.ImageFolder(root=data_dir, transform=transform)
loader = DataLoader(dataset, batch_size=16, shuffle=False)

mean = 0.
std = 0.
nb_samples = 0.

for data, _ in loader:
    batch_samples = data.size(0)
    data = data.view(batch_samples, data.size(1), -1)
    mean += data.mean(2).sum(0)
    std += data.std(2).sum(0)
    nb_samples += batch_samples

mean /= nb_samples
std /= nb_samples

print(f"Mean: {mean}")
print(f"Std: {std}")


Mean: tensor([0.5037, 0.4416, 0.4057])
Std: tensor([0.2471, 0.2342, 0.2316])


### Training model on training data

In [6]:
def get_flattened_size(model, input_size):
    dummy_input = torch.zeros(1, *input_size)
    with torch.no_grad():
        output = model.conv_layers(dummy_input)
    return output.numel()


class Net(nn.Module):

    def __init__(self):
        super().__init__()

        self.conv1 = nn.Conv2d(3, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.conv3 = nn.Conv2d(64, 128, 3, 1)

        self.bn1 = nn.BatchNorm2d(32)
        self.bn2 = nn.BatchNorm2d(64)
        self.bn3 = nn.BatchNorm2d(128)

        self.dropout1 = nn.Dropout(0.4)
        self.dropout2 = nn.Dropout(0.5)

        self.conv_layers = nn.Sequential(
            self.conv1,
            self.bn1,
            nn.ReLU(),
            nn.MaxPool2d(2),
            self.conv2,
            self.bn2,
            nn.ReLU(),
            nn.MaxPool2d(2),
            self.conv3,
            self.bn3,
            nn.ReLU(),
            nn.MaxPool2d(2),
            self.dropout1,
        )

        input_size = (3, 256, 256)
        self.fc1_input_features = get_flattened_size(self, input_size)

        self.fc1 = nn.Linear(self.fc1_input_features, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 2)

    def forward(self, x):
        x = self.conv_layers(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.fc3(x)
        output = F.log_softmax(x, dim=1)
        return output


def train(model, device, train_loader, optimizer, epoch, log_interval, dry_run):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)  
        optimizer.zero_grad()  
        output = model(data)  
        loss = F.nll_loss(output, target) 
        loss.backward()  
        optimizer.step() 
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
            if dry_run:
                break


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)  
            output = model(data) 
            test_loss += F.nll_loss(output, target, reduction='sum').item()
            pred = output.argmax(dim=1, keepdim=True) 
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset) 

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


def predict_image(model, device, image_path, transform, classes):
    model.eval()
    image = Image.open(image_path).convert('RGB')
    image = transform(image).unsqueeze(0).to(device)

    with torch.no_grad():
        output = model(image)
        pred = output.argmax(dim=1).item()

    return classes[pred]


def run(
        batch_size=16, 
        test_batch_size=16, 
        epochs=10,
        lr=0.0005,
        gamma=0.7,
        dry_run=False,
        seed=1,
        log_interval=10,
        save_model=True,
    ):

    torch.manual_seed(seed)

    device = "mps" if torch.backends.mps.is_available() else "cpu"
    print(f"Using device: {device}")
    
    train_kwargs = {'batch_size': batch_size, 'shuffle': True, 'pin_memory': False}
    test_kwargs = {'batch_size': test_batch_size, 'shuffle': False, 'pin_memory': False}

    transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
        transforms.Normalize(mean=(0.5037, 0.4416, 0.4057), std=(0.2471, 0.2342, 0.2316))

    ])

    data_dir = "../dataset_regular_or_sun"
    dataset = datasets.ImageFolder(data_dir, transform=transform)

    train_size = int(0.8 * len(dataset))
    test_size = len(dataset) - train_size
    train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

    train_loader = torch.utils.data.DataLoader(train_dataset, **train_kwargs)
    test_loader = torch.utils.data.DataLoader(test_dataset, **test_kwargs)

    model = Net().to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr)

    scheduler = StepLR(optimizer, step_size=3, gamma=gamma)
    for epoch in range(1, epochs + 1):
        train(model, device, train_loader, optimizer, epoch, log_interval, dry_run)
        test(model, device, test_loader)
        scheduler.step()

    if save_model:
        torch.save(model.state_dict(), "../models/NN_regular_or_sun.pt")


In [7]:
run()


Using device: mps
Train Epoch: 1 [0/816 (0%)]	Loss: 0.908812
Train Epoch: 1 [160/816 (20%)]	Loss: 2.768723
Train Epoch: 1 [320/816 (39%)]	Loss: 2.324268
Train Epoch: 1 [480/816 (59%)]	Loss: 1.656221
Train Epoch: 1 [640/816 (78%)]	Loss: 0.445060
Train Epoch: 1 [800/816 (98%)]	Loss: 0.694083

Test set: Average loss: 0.5588, Accuracy: 148/205 (72%)

Train Epoch: 2 [0/816 (0%)]	Loss: 0.567867
Train Epoch: 2 [160/816 (20%)]	Loss: 0.651550
Train Epoch: 2 [320/816 (39%)]	Loss: 0.493407
Train Epoch: 2 [480/816 (59%)]	Loss: 0.886345
Train Epoch: 2 [640/816 (78%)]	Loss: 0.538690
Train Epoch: 2 [800/816 (98%)]	Loss: 0.325902

Test set: Average loss: 0.4447, Accuracy: 167/205 (81%)

Train Epoch: 3 [0/816 (0%)]	Loss: 0.635746
Train Epoch: 3 [160/816 (20%)]	Loss: 0.434644
Train Epoch: 3 [320/816 (39%)]	Loss: 0.240579
Train Epoch: 3 [480/816 (59%)]	Loss: 0.275575
Train Epoch: 3 [640/816 (78%)]	Loss: 0.175137
Train Epoch: 3 [800/816 (98%)]	Loss: 0.268507

Test set: Average loss: 0.3429, Accuracy: 176/